### 2018/2019 - Task List 10

1. Implement Naive Bayes classifier with pyro
    - create apropriate parameters (mean and std for a and b, sigma - noise)
    - provide optimization procedure
    - check appropriateness of implemented method with selected dataset


# Required imports

In [1]:
%matplotlib inline
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive
from pyro.optim import Adam
import operator
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

In [2]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)

## Solutions

In [3]:
wine = pd.read_csv('wine.csv', header=None)
wine.columns = ["classname", "Alcohol", "MalicAcid", "Ash", "AlcalinityOfAsh", "Magnesium", "TotalPhenols","Flavanoids", 
                "NonflavanoidPhenols", "Proanthocyanins", "ColorIntensivity", "Hue","OD280/OD315", "Proline"]

df = wine
# num_columns = df.shape[1]
# num_rows = df.shape[0]
data = df

In [15]:
def model(data):
    data = torch.tensor(data.values).float()
    num_columns = data.shape[1]
    
    mean_prior = pyro.param("mean", data[0])
    std_prior = pyro.param("std", torch.ones(num_columns), constraint=constraints.positive)
#     noise_std = torch.ones(num_columns)
    
    prior = pyro.distributions.Normal(loc=mean_prior, scale=std_prior).independent(1)
    weight = pyro.sample("weight", prior)
    
    with pyro.plate("map", len(data)):
        #sample = pyro.sample("obs", pyro.distributions.Normal(weight, noise_std), obs=data)
        sample = pyro.sample("obs", prior, obs=data)
        return sample


def guide(data):
    num_columns = data.shape[1]
    
    mean = pyro.param("mean", torch.ones(1, num_columns)*0)
    std = pyro.param("std", torch.ones(1, num_columns)*1, constraint=constraints.positive)
    
    dists = pyro.distributions.Normal(loc=mean, scale=std).independent(1)
    sample = pyro.sample("weight", dists)  # , infer={'is_auxiliary': True})
    return sample


def train(data, num_steps=5000):
    pyro.clear_param_store()
    
    optim = Adam({"lr": 0.01})
    svi = pyro.infer.SVI(model=model,
                         guide=guide,
                         optim=optim,
                         loss=pyro.infer.Trace_ELBO(), num_samples=len(data))

    losses = []
    t = tqdm(range(num_steps))
    for j in t:
        loss = svi.step(data)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return pyro.param("mean"), pyro.param("std"), losses


def plot_loss(losses, learned_mean, learned_std, print_info=False):
    columns_data = [df[i] for i in df.columns]
    
    true_mean = [np.mean(x) for x in columns_data]
    true_std = [np.std(x) for x in columns_data]
    if (print_info):
        print(df.columns)
        print("{}\n{}".format(true_mean, true_std))
        print()

    plt.plot(losses)
    plt.title("evidence lower bound (ELBO)")
    plt.xlabel("step")
    plt.ylabel("loss");
    if (print_info):
        print('learned mean = ', learned_mean)
        print('learned std = ', learned_std)
        print()

    diff_mean = [learned_mean[i] - true_mean[i] for i in range(len(true_mean))]
    diff_std = [learned_std[i] - true_std[i] for i in range(len(true_std))]
    if (print_info):
        print(diff_mean, "\n", diff_std)

In [16]:
# %%time
# _, _, losses = train(data, 20000)

# learned_mean = pyro.param("mean").tolist()[0]
# learned_std = pyro.param("std").tolist()[0]

# plot_loss(losses, learned_mean, learned_std, print_info=True)

In [132]:
def extract_labels(dataset):
    # extract labels
    dataset_labels = dataset["classname"].copy()
    dataset = dataset.drop("classname", axis=1)

    return dataset, dataset_labels


def preprocess_data(X):
    # normalize [0,1]
    labels = X["classname"].copy()
    X = X.drop("classname", axis=1)
    columns = X.columns
    
    X = pd.DataFrame(normalize(X, axis=0), columns=columns)
    X = X.merge(labels, left_index=True, right_index=True)

    return X


def split_data(dataset, test_size=0.3):
    # split into train and test sets
    train_set, test_set = train_test_split(dataset, test_size=test_size, stratify=dataset['classname']) # random_state=42,

    # extract labels
    train_set_labels = train_set["classname"].copy()
    train_set = train_set.drop("classname", axis=1)

    test_set_labels = test_set["classname"].copy()
    test_set = test_set.drop("classname", axis=1)

    return train_set, train_set_labels, test_set, test_set_labels


def evaluate(labels_true, labels_predicted):
    # labels_true = labels_true.values.tolist()
    
    accuracy = metrics.accuracy_score(y_true=labels_true, y_pred=labels_predicted)
    precision = metrics.precision_score(y_true=labels_true, y_pred=labels_predicted, average='macro')
    recall = metrics.recall_score(y_true=labels_true, y_pred=labels_predicted, average='macro')
    f1 = metrics.f1_score(y_true=labels_true, y_pred=labels_predicted, average='macro')

    return accuracy, precision, recall, f1


def test_classifier(train, train_labels, test, test_labels, model):
    model.fit(train, train_labels)
    labels_predicted = model.predict(test)
    labels_true = test_labels
    
    temp_labels_t = []
    temp_labels_p = []
    
    for pred, true in zip(labels_predicted, labels_true):
        if pred != -1:
            temp_labels_t.append(true)
            temp_labels_p.append(pred)
    
    labels_true = temp_labels_t
    labels_predicted = temp_labels_p
    
    n = len(labels_true)
    
    matrix = confusion_matrix(labels_true, labels_predicted)
    accuracy, precision, recall, f1 = evaluate(labels_true, labels_predicted)
    return accuracy, precision, recall, f1, matrix, n

In [133]:
class NaiveBayesClassifier:
    def __init__(self, confidence_treshold=0.8, num_steps=10000):
        self.mean_for_classes = {}
        self.std_for_classes = {}
        self.classes_probs = {}
        self.classes = []
        self.confidence_treshold = confidence_treshold
        self.num_steps = num_steps
    
    def fit(self, X, y):
        num_columns = X.shape[1]
        self.classes = y.unique()
        
        for classname in self.classes:
            current_data = X[y==classname]
            self.classes_probs[classname] = len(current_data)/len(X)
            mean, std, _ = train(current_data, num_steps=self.num_steps)
            self.mean_for_classes[classname] = mean[0]
            self.std_for_classes[classname] = std[0]
            # print(self.classes_probs[classname], self.mean_for_classes[classname], self.std_for_classes[classname])
            
    
    def predict(self, X):
        probs = {}
        predicted = []
        
        for row in X.values:
            for classname in self.classes:
                p = self.classes_probs[classname]
                for i, element in enumerate(row):
                    mean = self.mean_for_classes[classname][i].detach().numpy()
                    std = self.std_for_classes[classname][i].detach().numpy()
                    p *= (1/(np.sqrt(2*np.pi*(std ** 2)))) * (np.e ** (-((element-mean) ** 2)/(2*std ** 2)))
                    # print(p)
                probs[classname] = p
                  
            probs = self.sum_to_one(probs)
            
            chosen_class = max(probs.items(), key=operator.itemgetter(1))[0]
            probability = probs[chosen_class]
#             print(probability)
            if probability >= self.confidence_treshold:
                predicted.append(chosen_class)
            else:
                predicted.append(-1)
        
        return predicted

    def sum_to_one(self, probs):
        s = sum(probs.values())
        if s == 0:
            s = 1
        for key in probs.keys():
            probs[key] = probs[key]/s
        return probs

In [134]:
def classify(dataset, num_steps=10000, confidence_treshold=0.8, show=False):
    #dataset = preprocess_data(dataset)
    if (show):
        columns_data = [dataset[i] for i in dataset.columns]
        true_mean = [np.mean(x) for x in columns_data]
        true_std = [np.std(x) for x in columns_data]
        print(dataset.columns)
        print("{}\n{}".format(true_mean, true_std))
        print()
    train_set, train_labels, test_set, test_labels = split_data(dataset)
    n_done = len(test_set)
    model = NaiveBayesClassifier(num_steps=num_steps, confidence_treshold=confidence_treshold)
    accuracy, precision, recall, f1, matrix, n_predicted = test_classifier(train_set, train_labels, test_set, test_labels, model)
    return accuracy, precision, recall, f1, matrix, n_predicted, n_done

In [136]:
%%time
accuracy, precision, recall, f1, matrix, n_predicted, n_done = classify(data, num_steps=10000, confidence_treshold=0.8, show=False)
print("acc: {}, pr: {}, rec: {}, f1: {}\nconfusion matrix:\n{}\npredicted {} out of {}".format(
    accuracy, precision, recall, f1, matrix, n_predicted, n_done))

acc: 1.0, pr: 1.0, rec: 1.0, f1: 1.0
confusion matrix:
[[15  0  0]
 [ 0 19  0]
 [ 0  0 15]]
predicted 49 out of 54
Wall time: 3min 47s
